# Combine supervised and unsupervised learning

In [1]:
import pandas as pd
from utils import Packet, Flow

def get_flows(df: pd.DataFrame, key_type: str = "default") -> dict:
    mp = dict()
    for idx in range(len(df)): # simulate the process of packet processing
        row = df.iloc[idx]
        pkt = Packet(
            src_ip=row["src_ip"],
            dst_ip=row["dst_ip"],
            src_port=row["src_port"],
            dst_port=row["dst_port"],
            protocol=row["protocol"],
            proto_code=row["proto_code"],
            pkt_length=row["pkt_length"],
            timestamp=row["timestamp"],
            ttl=row["ttl"],
            tcp_window=row["tcp_window"],
            tcp_dataoffset=row["tcp_dataoffset"],
            udp_length=row["udp_length"],
            label=row["label"],
        )
        key = pkt.key(type=key_type)
        if key not in mp:
            mp[key] = Flow()
        mp[key].add_packet(pkt)
    return mp

In [2]:
import torch
from config import whisper_config

def transform(mp: dict, feature_type: str = "whisper", 
              data_type: str = "train", test_data_aug: bool = True):
    packet_data, flow_data = [], []
    packet_labels, flow_labels = [], []
    for key, flow in mp.items():
        vec = flow.vector(feature_type=feature_type)
        if feature_type == "whisper":
            if len(vec) <= (whisper_config["n_fft"] // 2):
                # packet level features
                # vec = flow.packet_vector(agg_type="mean") + flow.packet_vector(agg_type="std") \
                #     + flow.packet_vector(agg_type="max") + flow.packet_vector(agg_type="min")
                # packet_data.append(vec)
                # packet_labels.append(flow.label)

                # implement fft on short flows
                ten = torch.tensor(vec)
                ten_fft = torch.fft.fft(ten, n=(whisper_config["n_fft"] // 2)+1)
                ten_power = torch.pow(ten_fft.real, 2) + torch.pow(ten_fft.imag, 2)
                ten_res = (ten_power.squeeze()+1).log2()
                ten_res = torch.where(torch.isnan(ten_res), torch.zeros_like(ten_res), ten_res)
                ten_res = torch.where(torch.isinf(ten_res), torch.zeros_like(ten_res), ten_res)
                if data_type == "test" and test_data_aug:
                    # data shape for test data augmentation: (n_flow, n_sample, floor(n_fft/2)+1)
                    packet_data.append([ten_res.tolist()])
                else:
                    # data shape for no data augmentation: (n_flow, floor(n_fft/2)+1)
                    packet_data.append(ten_res.tolist())
                packet_labels.append(flow.label)
                
            else:
                # flow level featrues
                ten = torch.tensor(vec)
                # stft requirement: input_size > (n_fft // 2)
                # default return shape: (floor(n_fft/2)+1, n_frame, 2)
                ten_fft = torch.stft(ten, whisper_config["n_fft"])
                ten_power = torch.pow(ten_fft[:,:,0], 2) + torch.pow(ten_fft[:,:,1], 2)
                ten_res = ((ten_power.squeeze()+1).log2()).permute(1,0)
                ten_res = torch.where(torch.isnan(ten_res), torch.zeros_like(ten_res), ten_res)
                ten_res = torch.where(torch.isinf(ten_res), torch.zeros_like(ten_res), ten_res)
                # ten_res shape: (n_frame, floor(n_fft/2)+1)
                if data_type == "train":
                    if (ten_res.size(0) > whisper_config["mean_win_train"]):
                        for _ in range(whisper_config["num_train_sample"]):
                            start_idx = torch.randint(0, ten_res.size(0)
                                        - whisper_config["mean_win_train"], (1,)).item()
                            ten_tmp = ten_res[start_idx:start_idx+whisper_config["mean_win_train"],:].mean(dim=0)
                            flow_data.append(ten_tmp.tolist())
                            flow_labels.append(flow.label)
                    else:
                        flow_data.append(ten_res.mean(dim=0).tolist())
                        flow_labels.append(flow.label)
                else: # for test
                    if test_data_aug:
                        tmp_data = []
                        if (ten_res.size(0) > whisper_config["mean_win_test"]):
                            # data augmentation for kmeans on flows with length > mean_win_test
                            for idx in range(0, ten_res.size(0) - whisper_config["mean_win_test"], 
                                            whisper_config["mean_win_test"]):
                                ten_tmp = ten_res[idx:idx+whisper_config["mean_win_test"],:].mean(dim=0)
                                tmp_data.append(ten_tmp.tolist())
                        else:
                            # no data augmentation for kmeans on flows with length < mean_win_test
                            tmp_data.append(ten_res.mean(dim=0).tolist())
                        flow_data.append(tmp_data)
                        # data shape for augmentation: (n_flow, n_sample, floor(n_fft/2)+1)
                    else: # for other detection methods
                        flow_data.append(ten_res.mean(dim=0).tolist())
                        # data shape for no augmentation: (n_flow, floor(n_fft/2)+1)
                    flow_labels.append(flow.label)
        elif feature_type == "encoding":
            # directly use the whisper encoding vector
            pass
        else: # for other feature types
            pass
    return packet_data, packet_labels, flow_data, flow_labels

/root/anaconda3/envs/PRO/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Mix benign and attack traffic

In [3]:
import numpy as np

def get_train_test_df(benign_path: str, attack_path: str):
    df_benign = pd.read_csv(benign_path)
    df_benign["label"] = 1
    df_attack = pd.read_csv(attack_path)
    df_attack["label"] = -1
    df_mix = pd.concat([df_benign, df_attack], ignore_index=True, axis=0)
    df_group = df_mix.groupby(["src_ip", "dst_ip", "src_port", "dst_port", "protocol"])
    df_mix["group"] = df_group.ngroup()
    df_mix["group"] = df_mix["group"] % 10
    df_mix["group"] = df_mix["group"].apply(lambda x: "train" if x < 6 else "test")
    # df_mix["group"].apply(lambda x: "train" if np.random.rand() < train_test_ratio else "test")
    df_train = df_mix[df_mix["group"] == "train"]
    df_test = df_mix[df_mix["group"] == "test"]
    return df_train, df_test

In [4]:
def get_mix_df(benign_path: str = None, 
                attack_path: str = None, 
                attack_ratio: int = None):
    assert benign_path is not None or attack_path is not None, \
        "benign_path and attack_path cannot be None at the same time"
    if benign_path is not None:
        df_benign = pd.read_csv(benign_path)
        df_benign["label"] = 1
    else:
        df_benign = None
    if attack_path is not None:
        df_attack = pd.read_csv(attack_path)
        df_attack["label"] = -1
        if attack_ratio is not None:
            df_group = df_attack.groupby(["src_ip", "dst_ip", "src_port", "dst_port", "protocol"])
            df_attack["group"] = df_group.ngroup()
            df_attack["group"] = df_attack["group"] % 500
            df_attack["group"] = df_attack["group"].apply(lambda x: "train" if x < attack_ratio else "test")
            df_attack = df_attack[df_attack["group"] == "train"]
    else:
        df_attack = None
    df_mix = pd.concat([df_benign, df_attack], ignore_index=True, axis=0)
    return df_mix

In [5]:
import os

benign_filenames = [os.path.join("train_set", "benign" + str(i) + ".csv") 
                    for i in range(1, 3)] + ["dataset_lite/mirai-benign.csv"]
attack_filenames = [os.path.join("dataset_lite", x+".csv") for x in 
                    ["BruteForce-Web", "BruteForce-XSS", "infiltration", 
                    "osscan", "SQL_Injection", "ssldosA10only", "mirai-attack"]]

In [82]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
import os
import json
import skops.io as sio

def train(data, labels, save_path, model="dt"):
    if model == "dt":
        clf = DecisionTreeClassifier(random_state=0)
    elif model == "svm":
        clf = SVC()
    else:
        clf = GaussianNB()
        # clf = MultinomialNB()
    clf.fit(data, labels)
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    sio.dump(clf, save_path)

from sklearn.metrics import accuracy_score

def test(data, labels, load_path, test_data_aug=False):
    clf = sio.load(load_path, True)
    if not test_data_aug:
        preds = clf.predict(data)
    else:
        preds = []
        for val in data:
            pred = clf.predict(val)
            preds.append(1 if sum(pred) > -1*len(pred) else -1)
    return accuracy_score(labels, preds)

In [86]:
USE_SHORT_FLOW = True

model = "svm"

if model == "dt":
    save_model_name = "decision_tree"
elif model == "svm":
    save_model_name = "svm"
else:
    save_model_name = "naive_bayes"

train_benign_filename = "dataset/benign_small.csv"
train_attack_filename = attack_filenames[-2]
# train_attack_filename = "none"

df_train = get_mix_df(benign_path=train_benign_filename, 
                    attack_path=train_attack_filename,
                    attack_ratio=1)
# df_train = get_mix_df(benign_path=train_benign_filename, 
#                     attack_path=None)
train_packet_data, train_packet_labels, train_flow_data, train_flow_labels \
    = transform(get_flows(df_train), data_type="train")

basename = "all" if USE_SHORT_FLOW else "long"
save_path = os.path.join("model", "whisper", save_model_name, 
            os.path.basename(train_benign_filename) + "_" + \
            os.path.basename(train_attack_filename) + "_" + basename + ".skops")

train_data = train_flow_data if not USE_SHORT_FLOW else train_flow_data + train_packet_data
train_labels = train_flow_labels if not USE_SHORT_FLOW else train_flow_labels + train_packet_labels
train(train_data, train_labels, save_path, model=model)

In [87]:
USE_DATA_AUG = True

detect_type = save_model_name if USE_DATA_AUG else save_model_name + "_no_aug"
accuracy_dict = {}

for test_benign_filename in benign_filenames:
    df_test = get_mix_df(benign_path=test_benign_filename)
    test_packet_data, test_packet_labels, test_flow_data, test_flow_labels \
    = transform(get_flows(df_test), data_type="test", test_data_aug=USE_DATA_AUG)
    test_data = test_flow_data if not USE_SHORT_FLOW else test_flow_data + test_packet_data
    test_labels = test_flow_labels if not USE_SHORT_FLOW else test_flow_labels + test_packet_labels
    acc = test(test_data, test_labels, save_path, test_data_aug=USE_DATA_AUG)
    print(f"accuracy of {test_benign_filename}: {acc}")
    accuracy_dict[test_benign_filename] = acc
    
for test_attack_filename in attack_filenames:
    df_test = get_mix_df(attack_path=test_attack_filename)
    test_packet_data, test_packet_labels, test_flow_data, test_flow_labels \
    = transform(get_flows(df_test), data_type="test", test_data_aug=USE_DATA_AUG)
    test_data = test_flow_data if not USE_SHORT_FLOW else test_flow_data + test_packet_data
    test_labels = test_flow_labels if not USE_SHORT_FLOW else test_flow_labels + test_packet_labels
    acc = test(test_data, test_labels, save_path, test_data_aug=USE_DATA_AUG)
    print(f"accuracy of {test_attack_filename}: {acc}")
    accuracy_dict[test_attack_filename] = acc

accuracy_base_name = "flow-accuracy.json" if not USE_SHORT_FLOW else "all-accuracy.json"
accuracy_save_path = os.path.join("result", "whisper", detect_type, 
                    os.path.basename(train_benign_filename)+"-"+os.path.basename(train_attack_filename),
                    accuracy_base_name)
os.makedirs(os.path.dirname(accuracy_save_path), exist_ok=True)
with open(accuracy_save_path, "w") as f:
    json.dump(accuracy_dict, f, indent=4)

accuracy of train_set/benign1.csv: 1.0
accuracy of train_set/benign2.csv: 1.0
accuracy of dataset_lite/mirai-benign.csv: 1.0
accuracy of dataset_lite/BruteForce-Web.csv: 0.0
accuracy of dataset_lite/BruteForce-XSS.csv: 0.0
accuracy of dataset_lite/infiltration.csv: 0.0
accuracy of dataset_lite/osscan.csv: 0.0
accuracy of dataset_lite/SQL_Injection.csv: 0.0
accuracy of dataset_lite/ssldosA10only.csv: 0.21739130434782608
accuracy of dataset_lite/mirai-attack.csv: 0.0


In [85]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# clf = GaussianNB()
clf = SVC()
X = [[1,2], [3,2], [5,1], [1,1], [2,1], [3,1]]
y = [1, 1, 1, 0, 0, 0]
clf.fit(X, y)
print(clf.predict([[1,1], [2,2], [3,3], [4,4], [5,5]]))

[0 1 1 1 1]


In [92]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
X = [[1,1], [2,2], [3,3]]
y = [1, 0, 0]
clf.fit(X, y)
print(clf.predict([[4,4], [5,5], [0,0]]))

X_ = [[4,4], [5,5], [0,0]]
y_ = [1, 1, 0]
clf.fit(X_, y_)

print(clf.predict([[1,1], [2,2], [3,3]]))

[0 0 1]
[0 0 1]


# Train ensemble

In [6]:
import torch.nn as nn

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Sigmoid())
        # self.decoder = nn.Sequential(
        #     nn.Linear(hidden_dim, input_dim),
        #     nn.Sigmoid())
        self.decoder = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [7]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        super(Dataset, self).__init__()
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [8]:
from sklearn.tree import DecisionTreeClassifier
import skops.io as sio
import os

def train_dt(train_data, train_labels, dt_save_path):
    clf = DecisionTreeClassifier(random_state=0)
    clf.fit(train_data, train_labels)
    if not os.path.exists(os.path.dirname(dt_save_path)):
        os.makedirs(os.path.dirname(dt_save_path))
    sio.dump(clf, dt_save_path)

In [9]:
from sklearn.cluster import KMeans
import torch

def train_kmeans(train_data, kmeans_save_path, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(train_data)
    centroids = torch.tensor(kmeans.cluster_centers_)
    train_loss = torch.cdist(torch.tensor(train_data), centroids, p=2).min(dim=1).values.mean()
    if not os.path.exists(os.path.dirname(kmeans_save_path)):
        os.makedirs(os.path.dirname(kmeans_save_path))
    with open(kmeans_save_path, "w") as f:
        json.dump({
            "centroids": centroids.tolist(),
            "train_loss": train_loss.item()
        }, f, indent=4)
    

In [10]:
import json

def train_ae(train_data, train_labels, save_dir,
            model, criterion, optimizer, device, 
            batch_size=32, num_epochs=200):
    train_dataset = Dataset(train_data, train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    loss_list = []
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        for data, labels in train_loader:
            # data = torch.sigmoid(data.to(device))
            data = data.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, data)
            loss_list.append(loss.item())
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")
    os.makedirs(save_dir, exist_ok=True)
    model_save_path = os.path.join(save_dir, "model.pt")
    torch.save(model.state_dict(), model_save_path)
    loss_save_path = os.path.join(save_dir, "train_loss.json")
    with open(loss_save_path, "w") as f:
        json.dump(loss_list, f)

In [11]:

def train_ensemble(benign_data, benign_labels, attack_data, attack_labels, 
        kmeans_save_path, n_clusters, dt_save_path, ae_save_dir,
        model, criterion, optimizer, device, batch_size=32, num_epochs=20
        ):
    train_dt(benign_data+attack_data, benign_labels+attack_labels, dt_save_path)
    train_kmeans(benign_data, kmeans_save_path, n_clusters=n_clusters)
    train_ae(benign_data, benign_labels, ae_save_dir, model, criterion, optimizer, 
             device, batch_size=batch_size, num_epochs=num_epochs)

In [12]:
def test_dt(data, dt_load_path):
    clf = sio.load(dt_load_path, True)
    dt_preds = []
    for val in data:
        pred = clf.predict(val)
        dt_preds.append(1 if sum(pred) > -1*len(pred) else -1)
    return dt_preds

In [13]:
def test_kmeans(data, kmeans_load_path, whisper_config, scale=10):
    with open(kmeans_load_path, "r") as f:
        model_param = json.load(f)
    centroids = torch.tensor(model_param["centroids"])
    train_loss = model_param["train_loss"]

    kmeans_preds, kmeans_ratios = [], []
    for val in data:
        val = torch.tensor(val)
        if (val.size(0) > whisper_config["mean_win_test"]):
            max_dist = 0
            for idx in range(0, val.size(0) - whisper_config["mean_win_test"], 
                             whisper_config["mean_win_test"]):
                ten_tmp = val[idx:idx+whisper_config["mean_win_test"],:].mean(dim=0)
                dist = torch.norm(ten_tmp - centroids, dim=1).min()
                max_dist = max(max_dist, dist)
            min_dist = max_dist
        else:
            min_dist = torch.norm(val.mean(dim=0) - centroids, dim=1).min()
        kmeans_preds.append(-1 if min_dist > scale * train_loss else 1)
        kmeans_ratios.append(min_dist/(scale * train_loss))
    return kmeans_preds, kmeans_ratios

In [14]:
def test_ae(test_data, model, device, criterion, 
            threshold, scale=5, test_data_aug=False):
    model.eval()
    preds = []
    with torch.no_grad():
        for val in test_data:
            # data = torch.sigmoid(torch.tensor(val).to(device))
            data = torch.tensor(val).to(device)
            outputs = model(data)
            loss = criterion(outputs, data)
            if not test_data_aug:
                preds.append(-1 if loss.item() > threshold * scale else 1)
            else:
                preds.append(-1 if loss.max().item() > threshold * scale else 1)
    return preds

In [18]:
from sklearn.metrics import accuracy_score

def test_ensemble(data, labels, kmeans_load_path, dt_load_path,
         model, device, criterion, threshold, whisper_config,
         kmeans_scale=10, ae_scale=1.5, test_data_aug=False):
    dt_preds = test_dt(data, dt_load_path)
    kmeans_preds, kmeans_ratios = test_kmeans(data, kmeans_load_path, whisper_config, scale=kmeans_scale)
    ae_preds = test_ae(data, model, device, criterion, threshold, scale=ae_scale, test_data_aug=test_data_aug)
    
    # preds = np.sign(np.array(dt_preds) + np.array(kmeans_preds) + np.array(ae_preds))
    preds = []
    for idx in range(len(kmeans_preds)):
        if kmeans_preds[idx] == -1 or ae_preds[idx] == -1 or dt_preds[idx] == -1:
            preds.append(-1)
        else:
            preds.append(1)
    return accuracy_score(labels, preds)

In [16]:
from config import whisper_config

USE_SHORT_FLOW = True

train_benign_filename = "dataset/benign_small.csv"
train_attack_filename = attack_filenames[-2]

dt_basename = "all" if USE_SHORT_FLOW else "long"
dt_save_path = os.path.join("model", "whisper", "decision_tree", 
            os.path.basename(train_benign_filename) + "_" + \
            os.path.basename(train_attack_filename) + "_" + dt_basename + ".skops")

kmeans_base_name = "flow-kmeans.json" if not USE_SHORT_FLOW else "all-kmeans.json"
kmeans_save_path = os.path.join("model", "whisper", "kmeans", 
                    os.path.basename(train_benign_filename), kmeans_base_name)

oe_save_dir = os.path.join("model", "whisper", "autoencoder", 
                        os.path.basename(train_benign_filename))

input_dim = whisper_config["n_fft"] // 2 + 1
hidden_dim = int(input_dim * 0.75)
model = AutoEncoder(input_dim, hidden_dim)
model.load_state_dict(torch.load(os.path.join(oe_save_dir, "model.pt")))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss()
model.to(device)
with open(os.path.join(oe_save_dir, "train_loss.json"), "r") as f:
    loss_list = json.load(f)
threshold = torch.tensor(loss_list).mean().item()

/root/anaconda3/envs/PRO/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [19]:
USE_DATA_AUG = True
detect_type = "ensemble" if USE_DATA_AUG else "ensemble-no-aug"
accuracy_dict = {}

for test_benign_filename in benign_filenames:
    test_df = pd.read_csv(test_benign_filename)
    test_df["label"] = 1
    test_packet_data, test_packet_lables, test_flow_data, test_flow_labels \
    = transform(get_flows(test_df), data_type="test", test_data_aug=USE_DATA_AUG)
    test_data = test_flow_data if not USE_SHORT_FLOW else test_flow_data + test_packet_data
    test_labels = test_flow_labels if not USE_SHORT_FLOW else test_flow_labels + test_packet_lables
    acc = test_ensemble(test_data, test_labels, kmeans_save_path, dt_save_path,
            model, device, criterion, threshold, whisper_config, test_data_aug=USE_DATA_AUG)
    print(f"accuracy of {test_benign_filename}: {acc}")
    accuracy_dict[test_benign_filename] = acc

for test_attack_filename in attack_filenames:
    test_df = pd.read_csv(test_attack_filename)
    test_df["label"] = -1
    test_packet_data, test_packet_lables, test_flow_data, test_flow_labels \
    = transform(get_flows(test_df), data_type="test", test_data_aug=USE_DATA_AUG)
    test_data = test_flow_data if not USE_SHORT_FLOW else test_flow_data + test_packet_data
    test_labels = test_flow_labels if not USE_SHORT_FLOW else test_flow_labels + test_packet_lables
    acc = test_ensemble(test_data, test_labels, kmeans_save_path, dt_save_path,
            model, device, criterion, threshold, whisper_config, test_data_aug=USE_DATA_AUG)
    print(f"accuracy of {test_attack_filename}: {acc}")
    accuracy_dict[test_attack_filename] = acc

accuracy_base_name = "flow-accuracy.json" if not USE_SHORT_FLOW else "all-accuracy.json"
accuracy_save_path = os.path.join("result", "whisper", detect_type, os.path.basename(train_benign_filename), accuracy_base_name)
os.makedirs(os.path.dirname(accuracy_save_path), exist_ok=True)
with open(accuracy_save_path, "w") as f:
    json.dump(accuracy_dict, f, indent=4)

accuracy of train_set/benign1.csv: 0.9630443467838594
accuracy of train_set/benign2.csv: 0.9627244340359095
accuracy of dataset_lite/mirai-benign.csv: 0.964148620265083
accuracy of dataset_lite/BruteForce-Web.csv: 0.9854014598540146
accuracy of dataset_lite/BruteForce-XSS.csv: 0.9767441860465116
accuracy of dataset_lite/infiltration.csv: 0.8333333333333334
accuracy of dataset_lite/osscan.csv: 0.007331378299120235
accuracy of dataset_lite/SQL_Injection.csv: 0.9411764705882353
accuracy of dataset_lite/ssldosA10only.csv: 0.8260869565217391
accuracy of dataset_lite/mirai-attack.csv: 0.03410001602820965
